In [31]:
# Import standard libraries
import os
import re
import sys
import glob
import shutil

## Import relevant libraries
import sys
import os
from natsort import natsorted  # For natural sorting
sys.path.append('../../activedrops')

## Including ourselves
import quadrop as qd
qd.set_plotting_style()





# Import data processing libraries
import pandas as pd
import numpy as np

# Import image processing libraries
import cv2
from PIL import Image, ImageEnhance, ImageOps
from scipy.ndimage import gaussian_filter

# Import plotting and visualization libraries
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import seaborn as sns
import colorcet as cc
from scipy.optimize import curve_fit
from ipywidgets import interact, FloatSlider

# Additional utilities
from natsort import natsorted  # For natural sorting
from sklearn.decomposition import PCA
import imageio.v2 as imageio

# Import plotting utilities
import cv2
import glob
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image, ImageDraw, ImageEnhance
import imageio.v2 as imageio
import seaborn as sns
from natsort import natsorted
from scipy.ndimage import gaussian_filter1d



In [32]:
# load PIV data from PIVlab into dataframes
def load_piv_data(data_path, condition, subcondition, min_frame=0, max_frame=None, skip_frames=1):
    """
    Processes Particle Image Velocimetry (PIV) data to create a DataFrame that combines mean values, 
    power calculations, and pivot matrices for each feature.

    Args:
        data_path (str): Path to the directory containing PIV data files.
        condition (str): Condition label for the data set.
        subcondition (str): Subcondition label for the data set.
        min_frame (int, optional): Minimum frame index to start processing (inclusive).
        max_frame (int, optional): Maximum frame index to stop processing (exclusive).

    Returns:
        pandas.DataFrame: A DataFrame where each row corresponds to a frame, combining mean values, 
        power calculations, and pivot matrices for each feature.
    """

    input_piv_data = os.path.join(data_path, condition, subcondition, "piv_data", "PIVlab_****.txt")
    
    # Using a for loop instead of list comprehension
    dfs = []
    for file in sorted(glob.glob(input_piv_data))[min_frame:max_frame:skip_frames]:
        df = pd.read_csv(file, skiprows=2).fillna(0).rename(columns={
            "magnitude [m/s]": "velocity magnitude [m/s]",
            "simple shear [1/s]": "shear [1/s]",
            "simple strain [1/s]": "strain [1/s]",
            "Vector type [-]": "data type [-]"
        })
        dfs.append(df)

    return dfs


In [33]:
data_path = "../../data/K401_Kif3-piv/"

# let's declare the variable "conditions" to the sorted list of subdirectories from data_path
conditions = natsorted([f for f in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, f))])

# let's declare the variable "subconditions" to the sorted list of subdirectories from the first condition
subconditions = natsorted([f for f in os.listdir(os.path.join(data_path, conditions[0])) if os.path.isdir(os.path.join(data_path, conditions[0], f))])

conditions

['k401', 'kif3']

In [34]:
df = load_piv_data(data_path, conditions[0], subconditions[0], min_frame=0, max_frame=None, skip_frames=1)
df

[          x [m]     y [m]  u [m/s]  v [m/s]  data type [-]  vorticity [1/s]  \
 0      0.000023  0.000023      0.0      0.0              2              0.0   
 1      0.000023  0.000045      0.0      0.0              2              0.0   
 2      0.000023  0.000067      0.0      0.0              2              0.0   
 3      0.000023  0.000089      0.0      0.0              2              0.0   
 4      0.000023  0.000111      0.0      0.0              2              0.0   
 ...         ...       ...      ...      ...            ...              ...   
 16124  0.002785  0.002697      0.0      0.0              2              0.0   
 16125  0.002785  0.002719      0.0      0.0              2              0.0   
 16126  0.002785  0.002741      0.0      0.0              2              0.0   
 16127  0.002785  0.002763      0.0      0.0              2              0.0   
 16128  0.002785  0.002785      0.0      0.0              2              0.0   
 
        velocity magnitude [m/s]  dive

In [35]:
df[300]

,x [m],y [m],u [m/s],v [m/s],data type [-],vorticity [1/s],velocity magnitude [m/s],divergence [1/s],dcev [1],shear [1/s],strain [1/s],vector direction [degrees]
0,0.000023,0.000023,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.000023,0.000045,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000023,0.000067,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000023,0.000089,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000023,0.000111,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
16124,0.002785,0.002697,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16125,0.002785,0.002719,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16126,0.002785,0.002741,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16127,0.002785,0.002763,0.0,0.0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [37]:
lambda_tau, results, fitted_values, intervector_distance_microns = qd.correlation_length(df[300])

0.00034555209226783666